# Stock Price Analysis: A Data-Driven Approach to Understanding Market Behavior

## Project Summary
This project demonstrates how data analysis can be used to evaluate stock market trends, compare performance across companies, and uncover actionable insights for strategic investment decisions. The analysis was conducted using Python and real-time data from public financial markets.



## Business Objective
To help stakeholders make better-informed investment or portfolio decisions by analyzing the historical performance, volatility, and risk-return profiles of selected stocks.

## Key Business Questions Addressed
* Which stocks delivered the best returns over time?
* How volatile are the selected companies, and what are the associated risks?
* What are the comparative trends and patterns in stock price behavior?
* Can we identify consistent performers or risky outliers?


## Approach

* Data Collection: Retrieved historical stock price data using the yfinance API.
* Data Cleaning & Preparation: Handled missing values, aligned time series, and calculated financial metrics.
* Comparative Analysis:Analyzed daily and cumulative returns across stocks.
* Identified volatility patterns using standard deviation and rolling averages.
* Visualized trends to reveal stock behavior over time.


# 1. Load Data Collection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [ ]:
read_list = glob.glob(r'/kaggle/input/stock-5yrs-analysis/*.csv')

In [ ]:
read_list[:5]

In [ ]:
company_list = [
    '/kaggle/input/stock-5yrs-analysis/AAPL_data.csv',
    '/kaggle/input/stock-5yrs-analysis/AMZN_data.csv',
    '/kaggle/input/stock-5yrs-analysis/GOOG_data.csv',
    '/kaggle/input/stock-5yrs-analysis/MSFT_data.csv'
    
]

In [ ]:
import warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
all_data = pd.DataFrame()

for file in company_list: 
    
    current_df = pd.read_csv(file)
    
    all_data = pd.concat([current_df, all_data])

In [ ]:
all_data.shape

In [ ]:
all_data.head(6)

In [ ]:
all_data['Name'].unique()

# 2. Analysing change in price of the stock overtime

In [ ]:
all_data.isnull().sum()

In [ ]:
all_data.dtypes

In [ ]:
all_data['date'] = pd.to_datetime(all_data['date'])

In [ ]:
all_data['date']

In [ ]:
tech_list = all_data['Name'].unique()

In [ ]:
tech_list

In [ ]:
plt.figure(figsize=(20,10))

for index, company in enumerate(tech_list, 1): 
    plt.subplot(2,2,index)
    filter1 = all_data['Name']==company
    df = all_data[filter1]
    plt.plot(df['date'], df['close'])
    plt.title(company)

# 3. Moving Average of Stocks

In [ ]:
all_data.head(5)

In [ ]:
all_data['close'].rolling(window=10).mean().head(14)

In [ ]:
new_data = all_data.copy()

In [ ]:
ma_day = [10, 20, 50]

for ma in ma_day:
    new_data['close'+str(ma)]= new_data['close'].rolling(ma).mean()

In [ ]:
new_data.tail(7)

In [ ]:
new_data.set_index('date', inplace=True)

In [ ]:
new_data.head(5)

In [ ]:
new_data.columns

In [ ]:
plt.figure(figsize=(20,15))

for index, company in enumerate(tech_list, 1): 
    plt.subplot(2,2,index)
    filter1 = new_data['Name']==company
    df = new_data[filter1]
    df[['close10','close20', 'close50']].plot(ax=plt.gca())
    plt.title(company)

# 4. Analyzing Closing price change in Amazon stock

In [ ]:
company_list

In [ ]:
amzn = pd.read_csv(r'/kaggle/input/stock-5yrs-analysis/AMZN_data.csv')

In [ ]:
amzn.head(4)

In [ ]:
amzn['close']

In [ ]:
amzn.head(4)

In [ ]:
amzn['Daily return(in %)'] = amzn['close'].pct_change() * 100

In [ ]:
amzn.head(4)

In [ ]:
import plotly.express as px

In [ ]:
px.line(amzn , x="date" , y="Daily return(in %)")

# 5. Performing resampling analysis of closing price

In [ ]:
amzn.dtypes

In [ ]:
amzn['date'] =pd.to_datetime(amzn['date'])

In [ ]:
amzn.dtypes

In [ ]:
amzn.head(4)

In [ ]:
amzn.set_index('date' , inplace=True)

In [ ]:
amzn.head(4)

In [ ]:
amzn['close'].resample('M').mean()

In [ ]:
amzn['close'].resample('M').mean().plot()

In [ ]:
amzn['close'].resample('Y').mean()

In [ ]:
amzn['close'].resample('Y').mean().plot() 

In [ ]:
amzn['close'].resample('Q').mean()

In [ ]:
amzn['close'].resample('Q').mean().plot()

# 6. Are the closing prices of the tech companies correlated?

In [ ]:
company_list

In [ ]:
company_list[0]

In [ ]:
app = pd.read_csv(company_list[0])
amzn = pd.read_csv(company_list[1])
google = pd.read_csv(company_list[2])
msft = pd.read_csv(company_list[3])

In [ ]:
closing_price = pd.DataFrame()

In [ ]:
closing_price['apple_close'] = app['close']
closing_price['amzn_close'] = amzn['close']
closing_price['goog_close'] = google['close']
closing_price['msft_close'] = msft['close']

In [ ]:
closing_price

In [ ]:
sns.pairplot(closing_price)

In [ ]:
closing_price.corr()

In [ ]:
sns.heatmap(closing_price.corr() , annot=True)

# 7. Correlation between Daily change in Closing price of stocks or Daily Returns in Stock

In [ ]:
closing_price

In [ ]:
closing_price['apple_close']

In [ ]:
closing_price['apple_close'].shift(1)

In [ ]:
(closing_price['apple_close'] - closing_price['apple_close'].shift(1))/closing_price['apple_close'].shift(1) * 100

In [ ]:
for col in closing_price.columns:
    closing_price[col + '_pct_change'] = (closing_price[col] - closing_price[col].shift(1))/closing_price[col].shift(1) * 100

In [ ]:
closing_price

In [ ]:
closing_price.columns

In [ ]:
clsing_p = closing_price[['apple_close_pct_change', 'amzn_close_pct_change',
       'goog_close_pct_change', 'msft_close_pct_change']]

In [ ]:
clsing_p

In [ ]:
g = sns.PairGrid(data= clsing_p)
g.map_diag(sns.histplot)
g.map_lower(sns.scatterplot)
g.map_upper(sns.kdeplot)

In [ ]:
clsing_p.corr()

In [ ]:
sns.heatmap(clsing_p.corr() , annot=True)